<a href="https://colab.research.google.com/github/Muhammad-Gunzalas/Machine-Learning-with-Naive-Bayes/blob/main/Multi_nomial_Naive_Bayes_from_scratch_on_Car_Evaluation_dataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.metrics as sk
import math
from collections import defaultdict

In [ ]:
data=pd.read_csv('/content/car_evaluation.csv',header=None)
data

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [ ]:
data.columns=['buying','maint','doors','persons','lug_boot','safety','class']

In [ ]:
data

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [ ]:
data['buying'].value_counts()

vhigh    432
high     432
med      432
low      432
Name: buying, dtype: int64

In [ ]:
data['maint'].value_counts()

vhigh    432
high     432
med      432
low      432
Name: maint, dtype: int64

In [ ]:
data['doors'].value_counts()

2        432
3        432
4        432
5more    432
Name: doors, dtype: int64

In [ ]:
data['persons'].value_counts()

2       576
4       576
more    576
Name: persons, dtype: int64

In [ ]:
data['lug_boot'].value_counts()

small    576
med      576
big      576
Name: lug_boot, dtype: int64

In [ ]:
data['safety'].value_counts()

low     576
med     576
high    576
Name: safety, dtype: int64

In [ ]:
data['class'].value_counts()

unacc    1210
acc       384
good       69
vgood      65
Name: class, dtype: int64

In [ ]:
#testing_data, training_data = np.split(data, [int(.3 * len(data))])
testing_data, training_data = np.split(data.sample(frac=1, random_state=1), [int(.3 * len(data))])


In [ ]:
testing_data.shape , training_data.shape

((518, 7), (1210, 7))

In [ ]:
training_data

,buying,maint,doors,persons,lug_boot,safety,class
422,vhigh,low,5more,4,big,high,acc
1497,low,high,5more,4,med,low,unacc
851,high,low,5more,4,med,high,acc
443,high,vhigh,2,4,small,high,unacc
1631,low,low,2,4,small,high,good
...,...,...,...,...,...,...,...
715,high,med,4,4,med,med,acc
905,med,vhigh,3,4,med,high,acc
1096,med,med,2,4,big,med,acc
235,vhigh,med,2,more,small,med,unacc


In [ ]:
testing_data

,buying,maint,doors,persons,lug_boot,safety,class
1233,med,low,3,more,small,low,unacc
592,high,high,3,more,big,med,acc
625,high,high,5more,2,med,med,unacc
1546,low,med,3,2,big,med,unacc
730,high,med,5more,2,small,med,unacc
...,...,...,...,...,...,...,...
1465,low,high,4,2,big,med,unacc
1443,low,high,3,4,med,low,unacc
490,high,vhigh,4,2,med,med,unacc
1032,med,high,4,2,big,low,unacc


In [ ]:
x_test=testing_data.drop('class',axis=1)
y_test=testing_data['class']

In [ ]:
x_test.shape , y_test.shape

((518, 6), (518,))

In [ ]:

def train_naive_bayes(training_data):
    # Calculate the prior probabilities
    class_counts = defaultdict(int)
    total_data_points = len(training_data)
    for _, data in training_data.iterrows():
        class_label = data['class']
        class_counts[class_label] += 1

    prior_probabilities = {}
    for class_label, count in class_counts.items():
        prior_probabilities[class_label] = count / total_data_points

    # Calculate the conditional probabilities
    conditional_probabilities = {}
    for class_label in class_counts:
        class_data = training_data[training_data['class'] == class_label]
        total_words_in_class = sum([len(features) for _, features in class_data.iloc[:, :-1].iterrows()])
        word_counts = defaultdict(int)
        for _, features in class_data.iloc[:, :-1].iterrows():
            for word in features:
                word_counts[word] += 1

        conditional_probabilities[class_label] = {}
        for word, count in word_counts.items():
            conditional_probabilities[class_label][word] = (count + 1) / (total_words_in_class + len(word_counts))

    return prior_probabilities, conditional_probabilities

def predict_naive_bayes(features, prior_probabilities, conditional_probabilities):
    class_scores = {}
    for class_label in prior_probabilities:
        score = math.log(prior_probabilities[class_label])
        for word in features:
            if word in conditional_probabilities[class_label]:
                score += math.log(conditional_probabilities[class_label][word])
            else:
                score += math.log(1 / len(conditional_probabilities[class_label]))
        class_scores[class_label] = score

    predicted_class_label = max(class_scores, key=class_scores.get)
    return predicted_class_label

In [ ]:

prior_probabilities, conditional_probabilities = train_naive_bayes(training_data)

predicted=[]

for _, features in x_test.iterrows():
    predicted_class_label = predict_naive_bayes(features, prior_probabilities, conditional_probabilities)
    predicted.append(predicted_class_label)

#print("Actual Label :",y_test.values)  
#print("Predicted Label :",predicted)

accuracy= sk.accuracy_score(y_test, predicted)


In [ ]:

print('Accuracy :',accuracy)
print('Precision:', sk.precision_score(y_test, predicted, average='macro'))
print('Recall:', sk.recall_score(y_test, predicted, average='macro'))
print('F1-score:', sk.f1_score(y_test, predicted, average='macro'))

Accuracy : 0.777992277992278
Precision: 0.6318555251276737
Recall: 0.35709541062801936
F1-score: 0.3732003260583512


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
